In [ ]:
##The dataset is loaded to your GDrive so need to be mounted
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/Satellite Project/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from utils import *

from models.basic_unet import UNet
from models.diff_unet import DiffUNet

from datasets.dataset import ChangeDetectionDataset
from datasets.diff_dataset import ChangeDetectionDiffDataset

## Read Data

In [41]:
model_name = 'DiffUNet.pt'
testset_path = './testset/'
predicted_images_dir = './testset/Predictions'

In [42]:
after_images = read_data(testset_path, 'after')
before_images = read_data(testset_path,'before')

Time taken to read  after  images: 0.01 seconds
Time taken to read  before  images: 0.01 seconds


In [43]:
diff_images = preprocess(before_images, after_images, save_dir=testset_path)

In [44]:
print(len(after_images))

1


In [45]:
batch_size = 30

In [46]:
# Define transformations
transform_rgb = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Assuming you have instantiated your dataset like this
test_dataset = ChangeDetectionDiffDataset(diff_images, diff_images, transform_rgb)
# test_dataset = ChangeDetectionDataset(before_images, after_images, before_images, transform_rgb)


# Define data loaders for train and validation sets
test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle = True)

In [47]:
# Create an instance of the model
threshold = 0.3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# must be changed for each model
model = UNet(n_channels = 3, n_classes = 1).to(device)
criterion = nn.BCEWithLogitsLoss()

print(device)

cpu


In [48]:
# Load the latest saved model
model = load_model(model, save_dir="saved_models", model_name=model_name, device=device)

In [49]:
# predicted_masks_list = predict(model, test_loader, device, threshold)
predicted_masks_list = predict_diff(model, test_loader, device, threshold)

In [50]:
save_predictions(predicted_masks_list)

Saving predictions
Saving Completed
